In [2]:
from datasets import load_dataset 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
import time
import pandas as pd
from datasets import load_dataset 

dataset = load_dataset("NicolaiSivesind/human-vs-machine", "wiki_labeled")
dataset = dataset['test'].select(range(10))

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-3b-wiki-detector", torchscript=True)
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-3b-wiki-detector", torchscript=True)
model.to("cpu")
print()

Loading checkpoint shards: 100%|██████████| 2/2 [01:27<00:00, 43.84s/it]

In [6]:
tokenized_texts = tokenizer(dataset['text'], padding=True, truncation = True, return_tensors="pt", max_length=512).to("cpu")
input_ids = tokenized_texts['input_ids'].to("cpu")
attention_mask = tokenized_texts['attention_mask'].to("cpu")

In [23]:
model.eval()
with torch.no_grad():
    start = time.time()
    outputs = model(input_ids, attention_mask = attention_mask)
    end = time.time()
    print("Time taken for 10 samples: ", end-start)

Time taken for 10 samples:  30.447028636932373


In [7]:
def inferenced_function(input_ids, attention_mask):
    return model(input_ids, attention_mask=attention_mask)

In [8]:
jit_base_model = torch.jit.trace(inferenced_function, (input_ids, attention_mask))

In [ ]:
start = time.time()
outputs = inferenced_function(input_ids, attention_mask)
end = time.time()
print("Time taken for 10 samples: ", end-start)

In [ ]:
torch.jit.save(traced_model, "bloomz_base_jit.pt")

In [ ]:
result = traced_model(encoded_dataset['validation']['input_ids'])

In [3]:
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

In [4]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 12010.384816
Size (MB): 4932.560034


In [12]:
tokenized_texts = [tokenizer(x['text'], padding=True, truncation=True, return_tensors="pt", max_length=512).to("cpu") for x in dataset]
time_taken = []
model = model.to("cpu")

for text in tokenized_texts:
    start_time = time.time()
    with torch.no_grad():
        logits = model(**text)
    end_time = time.time()
    time_taken.append(end_time - start_time)
output = pd.DataFrame({"times":time_taken})
output.to_csv("quantized_inference_time.csv", index=False)
display(output.describe())

,times
count,100.000000
mean,3.609569
std,1.427363
min,1.164556
25%,2.556017
50%,3.599232
75%,4.501231
max,9.016316
